In [73]:
import pandas as pd
train = pd.read_csv('data/laptop_train.csv')
test = pd.read_csv('data/laptop_test.csv')
train.head()

,Brand,Model,Series,Processor,Processor_Gen,RAM,Hard_Disk_Capacity,OS,Rating,Price
0,ASUS,VivoBook,15,i3,10th,8.0,512 GB SSD,Windows 11 Home,4.3,37940
1,DELL,Inspiron,NaN,i3,11th,8.0,1 TB HDD,Windows 11 Home,3.7,39040
2,ASUS,VivoBook,15,i7,10th,16.0,512 GB SSD,Windows 11 Home,4.1,57940
3,DELL,NaN,NaN,i3,10th,8.0,1 TB HDD,Windows 10,3.2,41340
4,Lenovo,IdeaPad,Slim,i3,11th,8.0,512 GB SSD,Windows 10 Home,4.4,45440


In [74]:
test.head()

,Brand,Model,Series,Processor,Processor_Gen,RAM,Hard_Disk_Capacity,OS,Rating
0,DELL,Vostro,NaN,i3,10th,8.0,256 GB SSD,Windows 10 Home,4.3
1,Lenovo,IdeaPad,3,i3,10th,8.0,256 GB SSD,Windows 11 Home,4.3
2,HP,NaN,NaN,i5,11th,8.0,512 GB SSD,Windows 11 Home,4.4
3,Lenovo,IdeaPad,3,i3,11th,8.0,256 GB SSD,Windows 11 Home,4.3
4,HP,250-G6,NaN,i5,7th,4.0,1 TB HDD,DOS,4.2


In [75]:
train.isna().sum()

Brand                  0
Model                  9
Series                36
Processor              5
Processor_Gen          5
RAM                    6
Hard_Disk_Capacity     6
OS                     6
Rating                 0
Price                  0
dtype: int64

In [76]:
test.isna().sum()

Brand                  0
Model                  5
Series                14
Processor              2
Processor_Gen          2
RAM                    2
Hard_Disk_Capacity     2
OS                     2
Rating                 0
dtype: int64

In [77]:
print(train.shape, test.shape)

(91, 10) (39, 9)


In [78]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Brand               91 non-null     object 
 1   Model               82 non-null     object 
 2   Series              55 non-null     object 
 3   Processor           86 non-null     object 
 4   Processor_Gen       86 non-null     object 
 5   RAM                 85 non-null     float64
 6   Hard_Disk_Capacity  85 non-null     object 
 7   OS                  85 non-null     object 
 8   Rating              91 non-null     float64
 9   Price               91 non-null     int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 7.2+ KB


In [79]:
train['Series'].unique()

array(['15', nan, 'Slim', 'Gaming', 'Ultra', 'Macbook', 'Ryzen', '3',
       '3405', 'Pavilion', 'Athlon', 'Celeron', 'TUF', '5', 'Duo', '14',
       'K15', '530s', 'Air', 'Thin', 'Dual'], dtype=object)

In [80]:
# do the model names match ?
set(train['Model'])==set(test['Model'])

False

In [81]:
# separat the price column and drop redundant ones
train_price = train['Price']
train = train.drop(columns=['Price', 'Series','Model'])
test = test.drop(columns=['Series','Model'])

# missing values in RAM <- -1 and others <- 'X'
train['RAM'] = train['RAM'].fillna(-1)
test['RAM'] = test['RAM'].fillna(-1)

train = train.fillna('X')
test = test.fillna('X')

print(train.isna().sum())
print(test.isna().sum())

Brand                 0
Processor             0
Processor_Gen         0
RAM                   0
Hard_Disk_Capacity    0
OS                    0
Rating                0
dtype: int64
Brand                 0
Processor             0
Processor_Gen         0
RAM                   0
Hard_Disk_Capacity    0
OS                    0
Rating                0
dtype: int64


In [82]:
# combine the two and apply the one-hot encoding
combined = pd.concat([train, test])
combined = pd.get_dummies(combined)
len(combined.keys())

56

In [83]:
# and split again
train = combined[:len(train)]
test = combined[-len(test):]

#check 
print(train.shape, test.shape)
print(len(train.keys()), len(test.keys()))

(91, 56) (39, 56)
56 56


In [84]:
set(train.keys())==set(test.keys())

True

In [85]:
from sklearn.model_selection import train_test_split
train_input, val_input, train_target, val_target = train_test_split(train, train_price)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train_input, train_target)
print(rf.score(train_input, train_target), rf.score(val_input, val_target))

0.9503578326389559 0.8794427011740493


In [86]:
pred= rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.head()

,pred
0,39965.540000
1,40906.406667
2,61752.380000
3,40687.756667
4,49872.650000


In [87]:
submit.to_csv('8_regression_notebook.csv', index=False)